In [ ]:
from twitter import *
from time import sleep
import sys
import os
import unicodedata
import pandas as pd
import json

In [ ]:
# Your Twitter App details
# Get them from http://dev.twitter.com
OAUTH_TOKEN = ""
OAUTH_SECRET = ""
CONSUMER_KEY = ""
CONSUMER_SECRET = ""

In [ ]:
twitter = Twitter(auth=OAuth(OAUTH_TOKEN, OAUTH_SECRET, CONSUMER_KEY, CONSUMER_SECRET), api_version="1.1")

In [ ]:
# Search keywords
keywords = ["fablab", "fab lab", "makerspace", "hackerspace", "techshop", "repaircafe", "repair cafe"]

In [ ]:
def search_labs_on_twitter(keywords):
    search_results = []
    members = []
    members_usernames = []
    members_data = {}
    search_results_stats = {}
    for word in keywords:
        search_results_word = []
        # Pagination issue https://twittercommunity.com/t/odd-pagination-behavior-with-get-users-search/148502
        # Only the first 1,000 matching results are available.
        # https://developer.twitter.com/en/docs/twitter-api/v1/accounts-and-users/follow-search-get-users/api-reference/get-users-search
        for i in range(50):
            try:
                search = twitter.users.search(q=word,page=i,count=20)
                search_results.append(search)
                search_results_word.append(search)
                for k in search:
                    search_results_word.append(k)
            except Exception as e:
                print(i-1, 'pages of results with', len(search_results_word),' results')
                break

    for result in search_results:
        for user in result:
            members_data[user["id"]] = user
        
    df = pd.DataFrame.from_dict(members_data)
    return df

In [ ]:
def twitter_labs_creation(df):
    df = df.T
    df.columns = df.iloc[0]
    df = df[1:]
    df = df.assign(creation_date = pd.to_datetime(df.loc[:, 'created_at'],errors ='coerce', utc=True))
    df.set_index(pd.DatetimeIndex(df['creation_date']))
    return df

In [ ]:
def create_twitter_labs_timeline(df, lab_type):
    df[lab_type] = 1
    return df.groupby(df["creation_date"].dt.year)[lab_type].sum()

In [ ]:
fablabs = search_labs_on_twitter(["fablab"])
makerspaces = search_labs_on_twitter(["makerspace"])
hackerspaces = search_labs_on_twitter(["hackerspace"])
techshops = search_labs_on_twitter(["techshop"])
repaircafes = search_labs_on_twitter(["repaircafe", "repair cafe"])

In [ ]:
fablabs.to_csv("data/twitter_fablabs.csv")
makerspaces.to_csv("data/twitter_makerspaces.csv")
hackerspaces.to_csv("data/twitter_hackerspaces.csv")
techshops.to_csv("data/twitter_techshops.csv")
repaircafes.to_csv("data/twitter_repaircafes.csv")

In [ ]:
fablabs_creation = twitter_labs_creation(fablabs)
makerspaces_creation = twitter_labs_creation(makerspaces)
hackerspaces_creation = twitter_labs_creation(hackerspaces)
techshops_creation = twitter_labs_creation(techshops)
repaircafes_creation = twitter_labs_creation(repaircafes)

In [ ]:
fablabs_timeline = create_twitter_labs_timeline(fablabs_creation, lab_type="Fab Labs")
makerspaces_timeline = create_twitter_labs_timeline(makerspaces_creation, lab_type="Makerspaces")
hackerspaces_timeline = create_twitter_labs_timeline(hackerspaces_creation, lab_type="Hackerspaces")
techshops_timeline = create_twitter_labs_timeline(techshops_creation, lab_type="Techshops")
repaircafes_timeline = create_twitter_labs_timeline(repaircafes_creation, lab_type="Repair Cafes")

In [ ]:
fablabs_timeline.to_csv("data/years_twitter_fablabs.csv")
makerspaces_timeline.to_csv("data/years_twitter_makerspaces.csv")
hackerspaces_timeline.to_csv("data/years_twitter_hackerspaces.csv")
techshops_timeline.to_csv("data/years_twitter_techshops.csv")
repaircafes_timeline.to_csv("data/years_twitter_repaircafes.csv")